In [1]:
# import the necessary packages
import cv2
from joblib import load
import numpy as np

In [11]:
# create a cascade classifier object
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprise"}

dt = load('data/testingtesting.joblib')

In [ ]:
def preprocess_image(image):
    # convert the image to grayscale
    gray_face = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # resize the image to 48x48 pixels
    face = cv2.resize(gray_face, (48, 48))
    # normalize the image
    #face = face / 255.0
    # reshape the image to 4D tensor
    face = face.reshape(1, 48*48)
    return face

In [12]:
# function to detect face
def detect_face(img):
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img)
    for (x,y,w,h) in face_rects:
        cv2.rectangle(face_img,(x,y),(x+w,y+h),(255,255,255),10)
        cropped_face = face_img[y:y+h, x:x+w]
        processed_face = preprocess_image(cropped_face)
        emotion = dt.predict(processed_face)
        print(emotion)
        
        emotion_label = emotion[0]
        cv2.putText(face_img, emotion_label, (x, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (33, 255, 239), 2)

    return face_img

In [13]:
# initialize the video stream and allow the camera sensor to warm up
cap = cv2.VideoCapture(0) # 0 is the default camera

while True:
    # Capture frame-by-frame, tuple unpacking
    ret, frame = cap.read(0) # ret is a boolean that returns true if the frame is available
    frame = detect_face(frame)
    cv2.imshow('frame', frame)  # display the frame
    if cv2.waitKey(1) & 0xFF == ord('q'):  # if 'q' is pressed, break the loop
        break

# When everything done, release the capture
cap.release()  # release the camera
cv2.destroyAllWindows()  # close the window